# Model training

## Import packages

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time

## Read data

In [25]:
df = pd.read_csv('data/out.csv')
## First move: 1 input 
df = df[['0', '1', '2']].agg(lambda x: ''.join(x.values), axis=1).T

In [26]:
array_of_poems = df.to_numpy()
array_of_poems

array(['last red in the skya small girls moon face risesover the counter',
       'christmas servicesa cellular phone rings outhandels messiah',
       'passover darkness before the buds burst opena childs eyes in death',
       ...,
       'aint no selfie onthis earth thats gonna make melike theresa may',
       'is doing a greatjob turning independentsinto democrats',
       'wanted to send aquick follow up on if theblood is loud talk soon'],
      dtype=object)

In [32]:
for i, sentence in enumerate(array_of_poems):
    if i == 10:
        break;
    print(sentence)
    

last red in the skya small girls moon face risesover the counter
christmas servicesa cellular phone rings outhandels messiah
passover darkness before the buds burst opena childs eyes in death
last night of summerthe bright full moon of last nighthidden by a cloud
midnight and full moonmy neighbour asks to borrowthe vacum cleaner
yellow walnut leavesslowly appear on the lawnearly morning light
after its first flightthe young gerfalcons talonstighter on my glove
sultry afternoononly the mailbox shadowcrosses the dirt road
long journey back home a forgotten bale of hayslowly rots away
autumn mist obscuresthe island in the distanceshe cleans her glasses
